# Import Libs & Data

In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2024-09-27 23:52:29.884678: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 23:52:29.890631: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 23:52:29.932100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 23:52:31.113546: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')

In [3]:
print(concrete_data.columns.values)

['Cement' 'Blast Furnace Slag' 'Fly Ash' 'Water' 'Superplasticizer'
 'Coarse Aggregate' 'Fine Aggregate' 'Age' 'Strength']


In [4]:
# We can easily check the dataset for any missing values by using the following one-liner:  
concrete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [5]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# Data Analysis 

In [6]:
# Remove the results from the dataset (which is the target data)
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] 
# The target data is the result
target = concrete_data['Strength'] 

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head() # The target data "strength"

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
# Also we can normalize the data by substracting the mean and dividing by the standard deviation.

# Z = (X - μ) / σ, its a standard score of the sample (Gaussian distribution)
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
# Let's save the number of predictors to n_cols since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

# Model Build

In [11]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=0)

# Display the shapes of the resulting datasets
print(f"Training predictors shape: {X_train.shape}")
print(f"Test predictors shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test target shape: {y_test.shape}")

Training predictors shape: (721, 8)
Test predictors shape: (309, 8)
Training target shape: (721,)
Test target shape: (309,)


## Three Hiden Layers

In [12]:
# Keras model

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [13]:
# build the model
model = regression_model()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# fit the model using only the training data
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)

Epoch 1/50


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1529.4061 - val_loss: 1529.1925
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1607.2960 - val_loss: 1509.2657
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1603.8904 - val_loss: 1495.0924
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1619.1694 - val_loss: 1484.0498
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1565.1782 - val_loss: 1473.5619
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1666.5474 - val_loss: 1462.0138
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1575.4851 - val_loss: 1448.5475
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1615.3945 - val_loss: 1431.8635
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1483.4983 - val_loss: 1411.0111
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1608.8143 - val_loss: 1384.7859
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1478.8325 - val_loss: 1351.7839
Epoch 12/50
16/16 

# Predict the results

In [15]:
# Predict the results for the test set
y_pred = model.predict(X_test)

# Display the predicted results
print(y_pred)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
[[56.838264]
 [50.77    ]
 [71.86961 ]
 [50.51165 ]
 [23.939976]
 [29.740662]
 [23.542143]
 [43.35452 ]
 [30.040565]
 [30.970036]
 [23.439064]
 [21.589474]
 [82.08193 ]
 [53.72546 ]
 [29.293968]
 [49.2038  ]
 [22.094112]
 [26.27401 ]
 [22.94409 ]
 [28.059484]
 [31.233646]
 [35.49632 ]
 [26.857367]
 [27.353733]
 [30.943554]
 [31.396618]
 [17.828018]
 [34.76565 ]
 [45.192375]
 [23.499819]
 [37.69175 ]
 [28.529911]
 [45.89626 ]
 [52.17784 ]
 [26.583328]
 [38.45947 ]
 [29.11868 ]
 [30.042318]
 [19.75818 ]
 [31.152487]
 [20.225262]
 [20.236567]
 [23.134422]
 [41.11272 ]
 [16.637253]
 [52.550556]
 [47.64408 ]
 [48.49611 ]
 [23.54048 ]
 [21.320345]
 [36.19998 ]
 [28.200308]
 [29.381449]
 [35.411438]
 [39.014427]
 [49.20096 ]
 [28.703684]
 [18.766727]
 [33.412193]
 [20.076014]
 [29.507244]
 [26.85891 ]
 [26.695503]
 [58.203552]
 [22.867086]
 [26.554523]
 [26.76384 ]
 [20.612663]
 [18.234587]
 [19.12445 ]
 [25.434937]
 [20.460995]
 [21.353477]
 [23.413164

In [16]:
# Compute the mean squared error between the predicted and actual concrete strength
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 158.70895264327007


# For the remaining 50-1 = 49 loops

In [17]:
# Initialize a list to store the mean squared errors
mse_list = [mse]

# Repeat the process 50 times
for i in range(1,50):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    # Build the model
    model = regression_model()
    
    # Fit the model using only the training data
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    
    # Predict the results for the test set
    y_pred = model.predict(X_test)
    
    # Compute the mean squared error between the predicted and actual concrete strength
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the mse to the list
    mse_list.append(mse)

# Calculate the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Mean of Mean Squared Errors: 141.2751275872068
Standard Deviation of Mean Squared Errors: 10.809540357498088
